# Convloution (합성곱)


(https://opentutorials.org/module/5268/29788)

특정한 패턴의 특징이 어디서 나타나는지 확인

conv 필터를 가지고 특징맵을 만들어 본다.

- 특징맵 (feature map)

<img src="./img/c2.png" width=70% height=70%>

- 컨볼루션 필터의 이해
  1. 필터는 **3차원 형태로 된 가중치**의 모음
  1. 필터는 앞선 레이어에서 생성된 **특징맵 전체**를 본다.
  1. **필터의 수만큼 특징맵**을 만들어 낸다.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
help(tf.keras.datasets.mnist.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.mnist:

load_data(path='mnist.npz')
    Loads the MNIST dataset.
    
    This is a dataset of 60,000 28x28 grayscale images of the 10 digits,
    along with a test set of 10,000 images.
    More info can be found at the
    [MNIST homepage](http://yann.lecun.com/exdb/mnist/).
    
    Args:
      path: path where to cache the dataset locally
        (relative to `~/.keras/datasets`).
    
    Returns:
      Tuple of NumPy arrays: `(x_train, y_train), (x_test, y_test)`.
    
    **x_train**: uint8 NumPy array of grayscale image data with shapes
      `(60000, 28, 28)`, containing the training data. Pixel values range
      from 0 to 255.
    
    **y_train**: uint8 NumPy array of digit labels (integers in range 0-9)
      with shape `(60000,)` for the training data.
    
    **x_test**: uint8 NumPy array of grayscale image data with shapes
      (10000, 28, 28), containing the test data. Pixel values range
      from

In [ ]:
### 영상 클론 코딩

# 1.과거의 데이터를 준비
(독립, 종속), _ = tf.keras.datasets.mnist.load_data()
독립 = 독립.reshape(60000, 28, 28, 1) # conv 는 3차원을 입력으로 받는다.
                                    # (6만장, 28 by 28 1장)
                                    # (28 ,28, 3) = 28 by 28 3차원
종속 = pd.get_dummies(종속)
print(독립.shpae, 종속.shape)

# 2. 모델의 구조를 만듭니다.
X = tf.keras.layers.Input(shape=[28, 28, 1]) # conv2d 는 3차원을 입력으로 받는다.
## 컨볼루젼 레이터 추가    (필터셋 몇개, 필터셋 사이즈)
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation='swish')(X) # 3개의 새 특징맵 = 3채널의 특징맵
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation='swish')(X) # 6개의 새 특징맵 = 6채널의 특징맵
H = tf.keras.layers.Flatten()(H) # 표로 만든다.
H = tf.keras.layers.Dense(84, acrivation='swish')(H)
Y = tf.keras.layers.Dense(10, activation='softmax')(H)
model = tf.keras.models.Model(X, Y)
model.complie(loss='categorical_corssentropy', merics='accuracy')

# 3. 데이터로 모델을 학습(FIT)합니다.
model.fit(독립, 종속, epochs=10)

# 4. 모델을 이용합니다.
print("Predictions : ", model.predict(독립[0:5]))


https://youtu.be/5eYHZ7wC3Qs

---

# 필터의 이해

1. 필터셋읜 **3차원 형태로 된 가충치**의 모음
2. 필터셋  하나는 앞선 레이어의 결과인 **"특징맵" 전체를 본다.** (입력되는 특징맵 전체)
3. 필터셋의 개수 만큼 특징 맵을 만든다.

- ex) 필터셋 3개 라고 하면, 가로선 필터, 세로선 필터, 대각선 필터 3개를 사용함.
- 각 필터들은 3차원 (흑백이라도 3차원으로 설정. (5,5,1) 이런식으로)

```python
Conv2D(3, kernel_size=5, activation='swish')
Conv2D(6, kernel_size=5, activation='swish')
```

---

### 필터 모형 

[F1][F2][F3].........[F1][F2][F3][F4][F5][F6]   

F1 : [(5,5,?)] # 개별 필터셋 하나가 3차원이다. 즉, 5,5 사이즈 필터가 여러 채널 존재  
F2 : [(5,5,?)] # 이때 ? 는 앞선 특징맵 전체의 필터 수와 같은 수가 들어간다.   
F3 : [(5,5,?)]  

- 흑백 이미지 (28, 28, 1) 이 입력되면, (5, 5, 1) / 전체 필터셋은 (5, 5, ?)
- 컬러 이미지 (32, 32, 3) 이 입력되면, (5, 5, 3) / 전체 필터셋은 (3, 5, 5, ?)

---

```python
Conv2D(3, kernel_size=5, activation='swish')
Conv2D(6, kernel_size=5, activation='swish')
```

<img src="./img/c5.png" width=70% height=70%>



```python
Conv2D(3, kernel_size=5, activation='swish')
```
- 숫자 4 (28, 28, 1)
- F1, F2, F3 : (5, 5, 1) 3개의 필터셋를 만든다. (컨볼루션 필터)
- 이 필터를 통해 3장의 특징맵을 만든다. : M1, M2, M3

```python
Conv2D(6, kernel_size=5, activation='swish')
```
- 앞서 만들어진 특징맵 3개를 : (24, 24, 3)
- F1, F2, F3, F4, F5, F6 : (5 by 5 , 3) 6개의 필터셋를 만든다. (차원을 맞춤) (컨볼루션 필터)
- 이때, F1-1 은 M1, F1-2 는 M2, F1-3은 M3 를 본다. 
- 이는 하나의 필터셋은 앞선 레이어의 특징맵 전체를 한번씩 모두 보는 것이다.
- 이 과정을 반복해 새로운 특징맵 M1 ~ M6 까지가 만들어 진다. (20, 20, 6)
- 필터셋이 6개인 6체널인 특징맵이 만들어 졌다.

컨볼루션 필터 : 이미지들이 0~9 중 어느 숫자인지 판단하기 위해 가장 좋은 특징맵 6개를 찾아라


---

## 참고 사항

<img src="./img/c3.png" width=70% height=70%>

<img src="./img/c4.png" width=70% height=70%>

---

In [1]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# 이미지를 3차원으로 reshape
x_train = x_train.reshape(-1, 28, 28, 1) # 계산하기 싫은 부분은 -1로 넣어주면 자동 계산이 된다. 
# reshape 은 60000 단위가 같아야 함. 이를 -1로 하면, 알아서 60000으로 생각함
x_test = x_test.reshape(-1, 28, 28, 1)

# 원핫인코딩
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)
(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


In [2]:
# 모델준비

X = tf.keras.Input(shape=[28, 28, 1])

H = tf.keras.layers.Conv2D(3, kernel_size=5, activation="swish")(X) # 3채널의 특징맵
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation="swish")(H) # 6채널의 특징맵

H = tf.keras.layers.Flatten()(H) # 표로 만든다.
H = tf.keras.layers.Dense(84, activation="swish")(H)
Y = tf.keras.layers.Dense(10, activation="softmax")(H)

model = tf.keras.Model(X, Y)
model.compile(loss="categorical_crossentropy", metrics="accuracy")

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
Total params: 203,068
Trainable params: 203,068
Non-trainable params: 0
_______________________________________________________

### 결과표 해석
```python

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
# 커널 사이즈가 5 이므로 (5,5) 필터를 통과해 사이즈가 (24, 24, 3) 로 줄었다.
# (24, 24, 3) 의 이미지가 6만장
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
# (5, 5) 필터를 통과해 사이즈가 (20, 20, 6) 로 줄었다.
# (20, 20) 의 표이므로, 400개의 숫자가 들어있다. 더불어 6채널 이므로, 2400개 숫자가 들어있다.
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
# 위 표를 한줄로 편다. (2400개의 입력을 받아서...)
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
# 84개의 특징을 뽑아 낸다.                   2400 + 1(bias) * 84 = 201684
# 수식이 84개라는 의미
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
# 최종적으로 10개의 특징을 뽑아 낸다.
# 10개의 수식 (1개의 수식을 위해서 84개의 가중치, 1개의 bias가 필요)
=================================================================
Total params: 203,068
# 컴퓨터가 찾은 가중치의 갯수 : 203,068
Trainable params: 203,068
Non-trainable params: 0
_________________________________________________________________

```

- 작은 수의 파라미터 + 다회 학습 (에포크) 이라면, GPU 를 사용해도 속도가 빨라지지 않는다.

In [12]:
2401*84


201684

In [7]:
model.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)

Epoch 1/10
480/480 [==============================] - 3s 5ms/step - loss: 0.0076 - accuracy: 0.9980 - val_loss: 0.1998 - val_accuracy: 0.9809
Epoch 2/10
480/480 [==============================] - 2s 5ms/step - loss: 0.0056 - accuracy: 0.9986 - val_loss: 0.1995 - val_accuracy: 0.9762
Epoch 3/10
480/480 [==============================] - 2s 5ms/step - loss: 0.0059 - accuracy: 0.9985 - val_loss: 0.3136 - val_accuracy: 0.9739
Epoch 4/10
480/480 [==============================] - 3s 5ms/step - loss: 0.0068 - accuracy: 0.9984 - val_loss: 0.2623 - val_accuracy: 0.9793
Epoch 5/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0057 - accuracy: 0.9986 - val_loss: 0.2671 - val_accuracy: 0.9812
Epoch 6/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0044 - accuracy: 0.9990 - val_loss: 0.2712 - val_accuracy: 0.9813
Epoch 7/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0049 - accuracy: 0.9988 - val_loss: 0.2445 - val_accuracy: 0.9810
Epoch 

In [8]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3055 - accuracy: 0.9799


[0.30545511841773987, 0.9799000024795532]

---
### 경사하강법 추가

loss가 가장 작은 global minimum 을 찾음을 증명 할 수 없다.  
우리가 찾는 것은 local minimum 이다. (결과가 매번 달라지는 이유). 이 weight 들도 충분이 쓸만 하기 때문에, 최적의 수준의 모델을 사용한다.

---

### Convolusion 연산

https://youtu.be/VHQbRI2Xvl0


<img src="./img/c6.png" width=70% height=70%>




필터를 통한 연산을 하고 나면, 결과의 사이즈가 작아지는 이유도 알 수 있다.
- 수식은 퍼셉트론의 수식 모양과 같다.
- 퍼셉트론에서는 최적의 특징을 추출, 컨볼루션에서는 최적의 특징맵을 추출

---

### 실제 계산 연습

<img src="./img/c7.png" width=70% height=70%>

- 숫자 4 이미지와, 가로선이 있는 필터

- 첫번째 칸의 계산  

|0|0|0|
|:-:|:-:|:-:|
|0|1|0|
|0|1|0|

|-1|-1|-1|
|:-:|:-:|:-:|
|2|2|2|
|-1|-1|-1|

- 0 + 0 + 0 + 0 + 2 + 0 + 0 + -1 + 0 = 1

---
위 계산을 반복해서 채우면  
(6 , 6)
|1|1|0|1|1|1|
|:-:|:-:|:-:|:-:|:-:|:-:|
|0|0|0|0|0|0|
|-1|-2|-3|-2|-2|-1|
|3|5|6|4|4|2|
|-2|-3|-3|-2|-2|-1|
|0|0|0|1|1|1|

<img src="./img/c8.png" width=30% height=30%>

---


- 위 모델에서는 자의적으로 가로선 필터를 만들었지만, 컨볼루션 레이어를 만들어 컴퓨터가 적절한 필터를 만들어 낸다. (핵심)

- 딥러닝은 이해가 깊어질수록 이해하기 어렵다!

---
---
---

# MaxPooling

### 결과표 해석 (2)
```python

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
# 커널 사이즈가 5 이므로 (5,5) 필터를 통과해 사이즈가 (24, 24, 3) 로 줄었다.
# (24, 24, 3) 의 이미지가 6만장
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
# (5, 5) 필터를 통과해 사이즈가 (20, 20, 6) 로 줄었다.
# (20, 20) 의 표이므로, 400개의 숫자가 들어있다. 더불어 6채널 이므로, 2400개 숫자가 들어있다.
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
# 위 표를 한줄로 편다. (2400개의 입력을 받아서...)
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
# 84개의 특징을 뽑아 낸다.                   2400 + 1(bias) * 84 = 201684
# 수식이 84개라는 의미
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
# 최종적으로 10개의 특징을 뽑아 낸다.
# 내가 가진 종속변수의 값이 10 (0 ~ 9 까지 이므로 10이다.)
# 10개의 수식 (1개의 수식을 위해서 84개의 가중치, 1개의 bias가 필요)
=================================================================
Total params: 203,068
# 컴퓨터가 찾은 가중치의 갯수 : 203,068
Trainable params: 203,068
Non-trainable params: 0
_________________________________________________________________

```

- Conv 를 통해 출력된 특징맵의 갯수가 증가하면 (20, 20, 6),
- flatten 이후 입력으로 사용할 칼럼수가 증가하고 (2400),
- 이는 컴퓨터가 찾아야 하는 가중치의 증가를 의미한다. (201684)

**그래서 도입된 방법이 Pooling 이다.**

---

In [ ]:
# 모델준비
X = tf.keras.Input(shape=[28, 28, 1])
H = tf.keras.layers.Conv2D(3, kernel_size=5, activation="swish")(X) # 3채널의 특징맵
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation="swish")(H) # 6채널의 특징맵
H = tf.keras.layers.Flatten()(H) # 표로 만든다.
H = tf.keras.layers.Dense(84, activation="swish")(H)
Y = tf.keras.layers.Dense(10, activation="softmax")(H)
model = tf.keras.Model(X, Y)
model.compile(loss="categorical_crossentropy", metrics="accuracy")

model.summary()

### 결과표 계산 

- 넣을 사진의 사이즈 (28, 28, 1)
- Conv 를 통해 < Conv2D(3, kernel_size=5, > 는 (5, 5, 1) 이 3장이라는 것을 안다.

```python

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 24, 3)         78        
# 입력 : (5, 5, 1)
# (5 * 5) + 1(b) * 3 = (25 + 1) * 3 = 78 
# 3채널이 되었으므로, 다음 conv 로는 3채널이 넘어감
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 6)         456       
# 입력 : (5, 5, 3)
# (5 * 5 * 3) + 1(b) * 6 = (75 + 1) * 6 = 456 
_________________________________________________________________
flatten (Flatten)            (None, 2400)              0         
# (20 * 20 * 6) = 2400
_________________________________________________________________
dense (Dense)                (None, 84)                201684    
# (2400 + 1) * 84 = 201684
_________________________________________________________________
dense_1 (Dense)              (None, 10)                850       
# (84 + 1) * 10 = 850
=================================================================
Total params: 203,068
# 컴퓨터가 찾은 가중치의 갯수 : 203,068
Trainable params: 203,068
Non-trainable params: 0
_________________________________________________________________

```

---

### Pooling 을 적용한 코드

In [ ]:
# 모델준비
X = tf.keras.Input(shape=[28, 28, 1])

H = tf.keras.layers.Conv2D(3, kernel_size=5, activation="swish")(X) 
H = tf.keras.layers.MaxPool2D()(H)
H = tf.keras.layers.Conv2D(6, kernel_size=5, activation="swish")(H) 
H = tf.keras.layers.MaxPool2D()(H)

H = tf.keras.layers.Flatten()(H)
H = tf.keras.layers.Dense(84, activation="swish")(H)
Y = tf.keras.layers.Dense(10, activation="softmax")(H)
model = tf.keras.Model(X, Y)
model.compile(loss="categorical_crossentropy", metrics="accuracy")

model.summary()

### 결과 해석

```python
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
input_4 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 6)           456       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 6)           0         
# pooling을 하고 나면, (4, 4) 이미지 6장을 사용한다.
_________________________________________________________________
flatten_3 (Flatten)          (None, 96)                0         
# (4 * 4 * 6) = 96
_________________________________________________________________
dense_6 (Dense)              (None, 84)                8148      
# 84개의 수식, 96개의 가중치, 1개의 바이오스
# (96 + 1) * 84 = 8148
_________________________________________________________________
dense_7 (Dense)              (None, 10)                850       
# (84 + 1) * 10 = 850
=================================================================
Total params: 9,532
Trainable params: 9,532
Non-trainable params: 0
_________________________________________________________________
```
- Maxpooling 을 사용하면 특징맵의 사이즈가 줄어들어, 컴퓨터가 찾아야 하는 가중치의 수가 줄어든다.
---

### Maxpooling

(6, 6) 사이즈의 특징맵을, 그 절반인 (3, 3) 사이즈로 줄인다. 

<img src="./img/c9.png" width=70% height=70%>

- (2, 2) 영역에서 가장 큰 수만 남긴다.

- conv로 찾은 특징들은 수치로 나타난다. 큰 수일수록 특징이 많음을 의미

- 큰수만 남기는 이유 : 특징맵에서 큰 수 = 필터로 찾으려는 특징이 많이 나타난 부분

---

### 데이터 실습

- 컨볼루션 필터가 찾아낸 특징 정보를 유지하면서 사이즈를 줄이기 위한 방법

In [15]:
# Maxpooling

import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

# 일단 y 값을 onehot 인코딩 하지 않고 넘어감 (자동으로 하는 방법)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [24]:
X = tf.keras.Input(shape=[28, 28, 1])

H1 = tf.keras.layers.Conv2D(3, kernel_size=5, activation="swish")(X) 
# kernel_size=3 이 일반적 (3, 3)
H1 = tf.keras.layers.MaxPool2D()(H1)
# mxapool 은 (2, 2) 가 디폴트 값. 옵션으로 조정 할 수 있음.

H2 = tf.keras.layers.Conv2D(6, kernel_size=5, activation="swish")(H1) 
H2 = tf.keras.layers.MaxPool2D()(H2)

H3 = tf.keras.layers.Flatten()(H2)
H3 = tf.keras.layers.Dense(84, activation="swish")(H3)
Y = tf.keras.layers.Dense(10, activation="softmax")(H3)

model = tf.keras.Model(X, Y)
model.compile(loss="sparse_categorical_crossentropy", metrics="accuracy")
# 로스값을 원레 "categorical_crossentropy" 을 사용 했는데, 위에서 onehot 인코딩을 안했다면,
# "sparse_categorical_crossentropy" 을 사용해 준다. (자동 onehot 인코딩)

model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 3)         78        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 12, 12, 3)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 6)           456       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 4, 4, 6)           0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 96)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 84)                8148

- 컨볼루션 레이어를 쓰는 이유는, 필터가 찾고 싶은 특징이 어디에 있는지 알고 싶다.
- 따라서 컨볼루션에서 높은 값은 아주 의미 있는 값
- conv 와 maxpooling 은 일반적으로 함께 사용한다.
- 또, 이미지 사이즈는 줄이되, 필터의 갯수는 늘리면서 학습하는게 좋다고 알려져 있다.

In [17]:
model.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)

Epoch 1/10
480/480 [==============================] - 5s 9ms/step - loss: 1.8044 - accuracy: 0.8060 - val_loss: 0.2415 - val_accuracy: 0.9286
Epoch 2/10
480/480 [==============================] - 4s 8ms/step - loss: 0.1860 - accuracy: 0.9454 - val_loss: 0.1633 - val_accuracy: 0.9524
Epoch 3/10
480/480 [==============================] - 4s 7ms/step - loss: 0.1186 - accuracy: 0.9644 - val_loss: 0.1106 - val_accuracy: 0.9688
Epoch 4/10
480/480 [==============================] - 4s 8ms/step - loss: 0.0917 - accuracy: 0.9711 - val_loss: 0.1032 - val_accuracy: 0.9718
Epoch 5/10
480/480 [==============================] - 4s 8ms/step - loss: 0.0770 - accuracy: 0.9762 - val_loss: 0.1040 - val_accuracy: 0.9709
Epoch 6/10
480/480 [==============================] - 4s 8ms/step - loss: 0.0653 - accuracy: 0.9794 - val_loss: 0.1128 - val_accuracy: 0.9700
Epoch 7/10
480/480 [==============================] - 4s 8ms/step - loss: 0.0575 - accuracy: 0.9822 - val_loss: 0.0885 - val_accuracy: 0.9745
Epoch 

In [18]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0799 - accuracy: 0.9789


[0.07989798486232758, 0.9789000153541565]

---

# LeNet

<img src="./img/c10.png" width=80% height=80%>

- 입력 이미지 : (32, 32)
- Conv 1 : (28, 28) * 6 이니, 사이즈가 4 줄었으니 (5, 5, 6) Conv 를 사용.
- Maxpo 1 : (14, 14) * 6 : 사이즈가 절반으로 줄어든다.
- Conv 2 : (10, 10) * 16 이니, 사이즈가 4 줄었으니 (5, 5, 16) Conv 를 사용.
- Maxpo 2 : (5, 5) * 16 : 사이즈가 절반으로 줄어든다.
- hidden 1 : 120개의 노드
- hidden 2 : 84개의 노드
- output : 10개의 출력

In [19]:
# 코드로 구현

import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28, 1) (60000,)
(10000, 28, 28, 1) (10000,)


In [26]:
X = tf.keras.Input(shape=[28, 28, 1])

H1 = tf.keras.layers.Conv2D(6, kernel_size=5, padding="same", activation="swish")(X)
# padding="same" 은 conv 결과인 특징맵의 사이즈가 입력 이미지와 동일한 사이즈로 출력된다.
# kernel 이 5 이므로, (28,28)을 받으면 (24, 24)로 줄어야 하는데, same 을 통해 (28, 28)이 유지된다.
H1 = tf.keras.layers.MaxPool2D()(H1)

H2 = tf.keras.layers.Conv2D(16, kernel_size=5, activation="swish")(H1)
H2 = tf.keras.layers.MaxPool2D()(H2)

H3 = tf.keras.layers.Flatten()(H2)
H3 = tf.keras.layers.Dense(120, activation="swish")(H3)
H3 = tf.keras.layers.Dense(84, activation="swish")(H3)
Y = tf.keras.layers.Dense(10, activation="softmax")(H3)

model = tf.keras.Model(X, Y)
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

model.summary()


Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 6)         156       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 120)               481

In [27]:
model.fit(x_train, y_train, epochs=10, batch_size=100, validation_split=0.2)

Epoch 1/10
480/480 [==============================] - 4s 7ms/step - loss: 0.9787 - accuracy: 0.8903 - val_loss: 0.1313 - val_accuracy: 0.9596
Epoch 2/10
480/480 [==============================] - 3s 7ms/step - loss: 0.0937 - accuracy: 0.9732 - val_loss: 0.0784 - val_accuracy: 0.9806
Epoch 3/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0632 - accuracy: 0.9820 - val_loss: 0.0914 - val_accuracy: 0.9763
Epoch 4/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0495 - accuracy: 0.9854 - val_loss: 0.0612 - val_accuracy: 0.9844
Epoch 5/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0410 - accuracy: 0.9884 - val_loss: 0.0645 - val_accuracy: 0.9831
Epoch 6/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0330 - accuracy: 0.9903 - val_loss: 0.0981 - val_accuracy: 0.9805
Epoch 7/10
480/480 [==============================] - 3s 6ms/step - loss: 0.0297 - accuracy: 0.9915 - val_loss: 0.0777 - val_accuracy: 0.9827
Epoch 

In [28]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.0897 - accuracy: 0.9819


[0.08967630565166473, 0.9818999767303467]